In [1]:
# import the necessary packages
import numpy as np
from imutils import face_utils
import dlib
import cv2
import math
import imutils
import face_recognition

from locations import landmarks

In [2]:
# initializing dlib's face detector and encoder, creating predictor
p = "../data/shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [3]:
def calc_angle(v1, v2):
    '''
    计算两个向量的夹角：
    AB = [5,-1,1,-3]
    CD = [4,1,4.5,4.5]
    print(angle_new(AB, CD),type(AB))
    '''
    dx1 = v1[2] - v1[0]
    dy1 = v1[3] - v1[1]
    dx2 = v2[2] - v2[0]
    dy2 = v2[3] - v2[1]
    angle1 = math.atan2(dy1, dx1)
    angle1 = int(angle1 * 180/math.pi)
    # print(angle1)
    angle2 = math.atan2(dy2, dx2)
    angle2 = int(angle2 * 180/math.pi)
    # print(angle2)
    if angle1*angle2 >= 0:
        included_angle = abs(angle1-angle2)
    else:
        included_angle = abs(angle1) + abs(angle2)
        if included_angle > 180:
            included_angle = 360 - included_angle
    return included_angle

def calc_wuguan_std(center_list):
    """
    输入一张脸在一个视频中的的五官中心点列表
    """
    if len(center_list) == 1:
        return np.array([0])
    elif len(center_list) > 1:
        return np.array([np.linalg.norm(np.array(center_list[i])-np.array(center_list[i+1]),axis=1).std() \
                for i in range(len(center_list)-1)])

In [41]:
# calc features
def feature_calc(video_path, n_frames=30):
    # Create video reader and find length
    v_cap = cv2.VideoCapture(video_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # pick n frames
    sample = np.linspace(0, v_len - 1, n_frames).astype(int)

    # by frame index
    face_color_avgs = []
    face_color_stds = []
    yanpi_face_avgs = []
    lip_color_avgs = []
    face_fluctuation = []
    brow_dists = []
    brow2nose_dists = []
    brow2eye_dists = []
    brow2nose_angles = []
    brow_thicks = []
    face_rects = []
    u_lip_thickness = []
    d_lip_thickness = []
    u_lip_width = []
    d_lip_width = []
    l_brow_length = []
    r_brow_length = []
    nose_width = []
    l_eye_width = []
    r_eye_width = []
    l_eye_height = []
    r_eye_height = []
    nose2ulip_dists = []
    xiaba2dlip_dists = []
    nose_color_avgs = []
    
    
    # part counter indexes
    r_eye_index = landmarks().get_polygons_index('R_EYE')
    l_eye_index = landmarks().get_polygons_index('L_EYE')
    xiaba_index = landmarks().get_polygons_index('XIABA')
    l_brow_index = landmarks().get_polygons_index('L_BROW')
    r_brow_index = landmarks().get_polygons_index('R_BROW')
    nose_index = landmarks().get_polygons_index('NOSE')
    u_lip_index = landmarks().get_polygons_index('U_LIP')
    d_lip_index = landmarks().get_polygons_index('D_LIP')
    
    latest_encodings = [] # init a list of the lastest encodings of each cluster of faces
    flag = 0 # set a detector flag in one video while looping
    
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
#             print('=========第{}帧========'.format(j))
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = imutils.resize(frame, width = int(frame.shape[1]/1.5))
            
            # detect faces in the grayscale image
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            
            # get encodings list of faces
            known_face_locations = [(rect.top(), rect.right(), rect.bottom(), rect.left()) for rect in rects]
            biden_encoding = face_recognition.face_encodings(face_image=frame, 
                                                             known_face_locations=known_face_locations)
                        
            for (i, rect) in enumerate(rects):
                f_encoding = biden_encoding[i] # the encoding of i-th face
                
                # find the face index of this rect update latest_encodings list
                if latest_encodings:
                    results = face_recognition.compare_faces(latest_encodings, f_encoding)
                    if sum(results) == 0:
                        flag = 1
                    else:
                        true_index = results.index(True)
                        latest_encodings[true_index] = f_encoding
                else:
                    flag = 1
                
                if flag:
                    latest_encodings.append(f_encoding)
                    true_index = len(latest_encodings) - 1
                    face_color_avgs.append([])
                    face_color_stds.append([])
                    yanpi_face_avgs.append([])
                    lip_color_avgs.append([])
                    face_fluctuation.append([])
                    brow_dists.append([])
                    brow2nose_dists.append([])
                    brow2eye_dists.append([])
                    brow2nose_angles.append([[],[]])
                    brow_thicks.append([])
                    face_rects.append([])
                    u_lip_thickness.append([])
                    d_lip_thickness.append([])
                    u_lip_width.append([])
                    d_lip_width.append([])
                    l_brow_length.append([])
                    r_brow_length.append([])
                    nose_width.append([])
                    l_eye_width.append([])
                    r_eye_width.append([])
                    l_eye_height.append([])
                    r_eye_height.append([])
                    nose2ulip_dists.append([])
                    xiaba2dlip_dists.append([])
                    nose_color_avgs.append([])
                    
                    flag = 0
                
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                    
                xia_lian = abs(shape[0][1] - shape[109][1])
                to_append_pnt = [[shape[0][0], int(shape[0][1] + xia_lian * 0.4)], 
                                   [shape[134][0], int(shape[134][1] + xia_lian * 0.4)]]

                # part counter points
                l_eye = shape[l_eye_index]
                r_eye = shape[r_eye_index]
                xiaba = np.concatenate((shape[xiaba_index], to_append_pnt), axis=0)
                u_lip = shape[u_lip_index]
                d_lip = shape[d_lip_index]
                l_brow = shape[l_brow_index]
                r_brow = shape[r_brow_index]
                nose = shape[nose_index]

                # make masks with coordinates
                mask_xiaba = np.zeros((frame.shape[0], frame.shape[1]))
                mask_eye = np.zeros((frame.shape[0], frame.shape[1]))
                mask_lip = np.zeros((frame.shape[0], frame.shape[1]))
                mask_nose = np.zeros((frame.shape[0], frame.shape[1]))

                cv2.fillPoly(mask_xiaba, [xiaba], 1)
                cv2.fillPoly(mask_eye, [l_eye, r_eye], 1)
                cv2.fillPoly(mask_lip, [u_lip, d_lip], 1)
                cv2.fillPoly(mask_nose, [nose], 1)

                mask_xiaba = mask_xiaba.astype(np.bool)
                xiaba_out = np.zeros_like(gray)
                xiaba_out[mask_xiaba] = gray[mask_xiaba]
                mask_eye = mask_eye.astype(np.bool)
                eye_out = np.zeros_like(gray)
                eye_out[mask_eye] = gray[mask_eye]
                mask_lip = mask_lip.astype(np.bool)
                lip_out = np.zeros_like(gray)
                lip_out[mask_lip] = gray[mask_lip]
                mask_nose = mask_nose.astype(np.bool)
                nose_out = np.zeros_like(gray)
                nose_out[mask_nose] = gray[mask_nose]

                l_x, l_y, l_w, l_h = cv2.boundingRect(l_eye)
                l_eye_rectangle = gray[int(l_y-0.25*l_h):l_y+l_h, l_x:l_x+l_w]
                r_x, r_y, r_w, r_h = cv2.boundingRect(r_eye)
                r_eye_rectangle = gray[int(r_y-0.25*r_h):r_y+r_h, r_x:r_x+r_w]

                # distances
                center = [l_brow.mean(axis=0).tolist(),
                          r_brow.mean(axis=0).tolist(),
                          l_eye.mean(axis=0).tolist(),
                          r_eye.mean(axis=0).tolist(),
                          nose.mean(axis=0).tolist(),
                          np.concatenate((u_lip,d_lip)).mean(axis=0).tolist()]
                brow_dist = np.linalg.norm(shape[70] - shape[82])
                brow2nose_dist = np.linalg.norm(np.concatenate((shape[70],shape[82])).mean(axis=0) - shape[143])
                brow2eye_dist = (np.linalg.norm(np.array(center[0])-np.array(center[2])) + np.linalg.norm(np.array(center[1])-np.array(center[3]))) / 2
                brow_thick = (np.linalg.norm(shape[96] - shape[110]) + np.linalg.norm(shape[74] - shape[88])) / 2
                u_lip_dist = np.linalg.norm(shape[159] - shape[187])
                d_lip_dist = np.linalg.norm(shape[19] - shape[174])
                u_lip_width_dist = np.linalg.norm(shape[152] - shape[165])
                d_lip_width_dist = np.linalg.norm(shape[25] - shape[11])
                l_brow_len_dist = np.linalg.norm(shape[103] - shape[92])
                r_brow_len_dist = np.linalg.norm(shape[70] - shape[81])
                nose_width_dist = np.linalg.norm(shape[139] - shape[147])
                l_eye_width_dist = np.linalg.norm(shape[59] - shape[48])
                r_eye_width_dist = np.linalg.norm(shape[26] - shape[38])
                l_eye_height_dist = np.linalg.norm(shape[55] - shape[64])
                r_eye_height_dist = np.linalg.norm(shape[31] - shape[44])
                nose2ulip_dist = np.linalg.norm(nose.mean(axis=0) - u_lip.mean(axis=0))
                xiaba2dlip_dist = np.linalg.norm(shape[xiaba_index].mean(axis=0) - d_lip.mean(axis=0))

                # angles
                nose_vec = np.array([shape[139], shape[147]]).mean(axis=0).tolist() + np.array([shape[135], shape[151]]).mean(axis=0).tolist()
                l_brow_vec = shape[103].tolist() + shape[113].tolist()
                r_brow_vec = shape[81].tolist() + shape[91].tolist()
                l_brow2nose_angle = calc_angle(nose_vec, l_brow_vec)
                r_brow2nose_angle = calc_angle(nose_vec, r_brow_vec)


                # calc feature and drop them into corresponding face index
                face_color_avg = xiaba_out.sum() / mask_xiaba.sum()
                face_color_avgs[true_index].append(face_color_avg)
                face_color_stds[true_index].append(xiaba_out.std())
                yanpi_avg = (l_eye_rectangle.sum() + r_eye_rectangle.sum() - eye_out.sum()) / ((l_eye_rectangle.shape[0] + \
                        r_eye_rectangle.shape[0]) * (l_eye_rectangle.shape[1] + r_eye_rectangle.shape[1]) - mask_eye.sum())
                yanpi_face_avgs[true_index].append(yanpi_avg)
                lip_color_avg = lip_out.sum() / mask_lip.sum()
                lip_color_avgs[true_index].append(lip_color_avg)
                face_fluctuation[true_index].append(center)
                brow_dists[true_index].append(brow_dist)
                brow2nose_dists[true_index].append(brow2nose_dist)
                brow2eye_dists[true_index].append(brow2eye_dist)
                brow2nose_angles[true_index][0].append(l_brow2nose_angle)
                brow2nose_angles[true_index][1].append(r_brow2nose_angle)
                brow_thicks[true_index].append(brow_thick)
                u_lip_thickness[true_index].append(u_lip_dist)
                d_lip_thickness[true_index].append(d_lip_dist)
                u_lip_width[true_index].append(u_lip_width_dist)
                d_lip_width[true_index].append(d_lip_width_dist)
                l_brow_length[true_index].append(l_brow_len_dist)
                r_brow_length[true_index].append(r_brow_len_dist)
                nose_width[true_index].append(nose_width_dist)
                l_eye_width[true_index].append(l_eye_width_dist)
                r_eye_width[true_index].append(r_eye_width_dist)
                l_eye_height[true_index].append(l_eye_height_dist)
                r_eye_height[true_index].append(r_eye_height_dist)
                nose2ulip_dists[true_index].append(nose2ulip_dist)
                xiaba2dlip_dists[true_index].append(xiaba2dlip_dist)
                nose_color_avg = nose_out.sum() / mask_nose.sum()
                nose_color_avgs[true_index].append(nose_color_avg)
                face_rects[true_index].append(rect.left())

    v_cap.release() # release video
    
    face_cnt = len(face_color_avgs) # 脸的数量
    feature_face_color = [[np.array(face_color_avgs[i]).std()] for i in range(face_cnt)] # 面部色度标准差
    feature_face_maxdiff = [[max(face_color_stds[i])] for i in range(face_cnt)] # 面部色度最大差异（最大标准差）
    feature_yanpi_face_diff = [[np.array(yanpi_face_avgs[i]).std()] for i in range(face_cnt)] # 眼皮面部色度差的标准差
    feature_lip_color = [[np.array(lip_color_avgs[i]).std()] for i in range(face_cnt)] # 嘴唇色度标准差
    feature_face_fluctuation = [[max(calc_wuguan_std(face_fluctuation[i]))] for i in range(face_cnt)] # 面部特征移动相对距离
    feature_brow_dist = [[np.array(brow_dists[i]).std()] for i in range(face_cnt)] # 眉毛距离标准差
    feature_brow2nose_dist = [[np.array(brow2nose_dists[i]).std()] for i in range(face_cnt)] # 眉心到鼻尖距离标准差
    feature_brow2eye_dist = [[np.array(brow2eye_dists[i]).std()] for i in range(face_cnt)] # 眉毛到眼睛距离标准差
    feature_brow2nose_angle = [[np.array(brow2nose_angles[i][0]).std(),np.array(brow2nose_angles[i][1]).std()] 
                               for i in range(face_cnt)] # 眉毛与鼻子夹角，左右眉毛都做了计算
    feature_brow_thick = [[np.array(brow_thicks[i]).std()] for i in range(face_cnt)] # 眉毛粗细
    feature_ulip_thickness = [[np.array(u_lip_thickness[i]).std()] for i in range(face_cnt)] # 上嘴唇厚度标准差
    feature_dlip_thickness = [[np.array(d_lip_thickness[i]).std()] for i in range(face_cnt)] # 下嘴唇厚度标准差
    feature_ulip_width = [[np.array(u_lip_width[i]).std()] for i in range(face_cnt)] # 上嘴唇宽度标准差
    feature_dlip_width = [[np.array(d_lip_width[i]).std()] for i in range(face_cnt)] # 下嘴唇宽度标准差
    feature_lbrow_len = [[np.array(l_brow_length[i]).std()] for i in range(face_cnt)] # 上眉毛长度标准差
    feature_rbrow_len = [[np.array(r_brow_length[i]).std()] for i in range(face_cnt)] # 下眉毛长度标准差
    feature_nose_width = [[np.array(nose_width[i]).std()] for i in range(face_cnt)] # 鼻子宽度标准差
    feature_leye_width = [[np.array(l_eye_width[i]).std()] for i in range(face_cnt)] # 左眼宽度标准差
    feature_reye_width = [[np.array(r_eye_width[i]).std()] for i in range(face_cnt)] # 右眼宽度标准差
    feature_leye_height = [[np.array(l_eye_height[i]).std()] for i in range(face_cnt)] # 左眼高度标准差
    feature_reye_height = [[np.array(r_eye_height[i]).std()] for i in range(face_cnt)] # 右眼高度标准差
    feature_nose2ulip_dist = [[np.array(nose2ulip_dists[i]).std()] for i in range(face_cnt)] # 鼻子中心到上唇中心的距离标准差
    feature_xiaba2dlip_dist = [[np.array(xiaba2dlip_dists[i]).std()] for i in range(face_cnt)] # 下唇中心到下颚中心的距离标准差
    feature_nose_color = [[np.array(nose_color_avgs[i]).std()] for i in range(face_cnt)] # 鼻子区域的颜色标准差
    
    rect_left = [[np.array(face_rects[i]).mean()] for i in range(face_cnt)] # 计算每张脸与左边框的距离，以此来打标签

    feature = []
    for i in range(face_cnt):
        feature.append(feature_face_color[i] +
                       feature_face_maxdiff[i] +
                       feature_yanpi_face_diff[i] +
                       feature_lip_color[i] +
                       feature_face_fluctuation[i] + 
                       feature_brow_dist[i] + 
                       feature_brow2nose_dist[i] +
                       feature_brow2eye_dist[i] +
                       feature_brow2nose_angle[i] +
                       feature_brow_thick[i] + 
                       feature_ulip_thickness[i] + 
                       feature_dlip_thickness[i] + 
                       feature_ulip_width[i] + 
                       feature_dlip_width[i] + 
                       feature_lbrow_len[i] + 
                       feature_rbrow_len[i] + 
                       feature_nose_width[i] + 
                       feature_leye_width[i] + 
                       feature_reye_width[i] + 
                       feature_leye_height[i] + 
                       feature_reye_height[i] + 
                       feature_nose2ulip_dist[i] + 
                       feature_xiaba2dlip_dist[i] + 
                       feature_nose_color[i] + 
                       rect_left[i])
    print(feature)
    return feature

In [42]:
def main():
    return feature_calc('../data/exbxfmqqpx.mp4')

In [43]:
import time
t1 = time.time()
main()
print(time.time()-t1)

[[2.9528528821054545, 4.9891731774863946, 0.8805247890338259, 4.186178717744759, 6.132510285414462, 2.4974263964762273, 7.873342167384229, 1.8648729108092956, 2.8973363324145254, 3.048363004434138, 0.49287265138035974, 0.5661290467125353, 0.7076418763172199, 2.723017973112768, 2.0895342526337957, 3.094565500243091, 2.6292673104584403, 1.140423063681016, 1.0943632433096537, 1.6550155651693033, 0.9046750748700084, 0.9746464120370437, 1.2516412767270821, 4.307534649508412, 3.9471592534362108, 833.4761904761905], [0.0, 7.9334629969536214, 0.0, 0.0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 233.0]]
5.789464950561523


In [ ]:
import pandas as pd
import json
import os
import time

def calc_folder(folder_num):
    cnt = 0
    out_path = "/home/sufedc_nvidia_handianyouhua/deep_fake/results/"
    data_path = "/home/sufedc_nvidia_handianyouhua/deep_fake/data/data/"
    metadata = json.loads(open(os.path.join(data_path, 'dfdc_train_part_{}'.format(folder_num), 'metadata.json')).read())
    features = []
    t1 = time.time()

    for video in metadata.keys():
        if cnt % 20 == 0:
            print('============{} FINISHED, {}min used========='.format(cnt, (time.time()-t1) / 60.0))   
        cnt += 1
        video_path = os.path.join(data_path, 'dfdc_train_part_{}'.format(folder_num), video)
        print(video)
        if os.path.exists(video_path):
            if metadata[video]['label'] == 'FAKE':
                label = 0
            else:
                label = 1
            feature = feature_calc(video_path, n_frames=20)
            if feature != []:
                for i in range(len(feature)):
                    features.append([video] + feature[i] + [label])

    res = pd.DataFrame(features,columns = ['video',
                                           'face_color',
                                           'face_maxdiff',
                                           'yanpi_face_diff',
                                           'lip_color',
                                           'face_fluctuation',
                                           'brow_dist',
                                           'brow2nose_dist',
                                           'brow2eye_dist',
                                           'l_brow2nose_angle',
                                           'r_brow2nose_angle',
                                           'brow_thick',
                                           'ulip_thickness',
                                           'dlip_thickness',
                                           'ulip_width',
                                           'dlip_width',
                                           'lbrow_len',
                                           'rbrow_len',
                                           'nose_width',
                                           'leye_width',
                                           'reye_width',
                                           'leye_height',
                                           'reye_height',
                                           'nose2ulip_dist',
                                           'xiaba2dlip_dist',
                                           'nose_color',
                                           'rect_left',
                                           'label'])

    res.to_csv(out_path+'dfdc_train_part_{}.csv'.format(folder_num), sep=',', index=False, header=True)